In [236]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [237]:
#read spotify csv file
spotify_file = "spotify.csv"
spotify_df = pd.read_csv(spotify_file, engine = "python", header = 1)
spotify_df.head()

,Position,Track Name,Artist,Streams,URL
0,1,Dance Monkey,Tones And I,47797217,https://open.spotify.com/track/1rgnBhdG2JDFTbY...
1,2,All I Want for Christmas Is You,Mariah Carey,45828931,https://open.spotify.com/track/0bYg9bo50gSsH3L...
2,3,Last Christmas,Wham!,37725693,https://open.spotify.com/track/2FRnf9qhLbvw8fu...
3,4,ROXANNE,Arizona Zervas,32309199,https://open.spotify.com/track/696DnlkuDOXcMAn...
4,5,Santa Tell Me,Ariana Grande,27506141,https://open.spotify.com/track/0lizgQ7Qw35od7C...


### Create new data with select columns

In [238]:
#transform spotify_df to match tables
spotify_df = spotify_df[["Track Name", "Artist", "Streams"]].copy()
spotify_df = spotify_df.rename(columns={'Track Name': 'track_name', 'Artist': 'artist', 'Streams': 'streams'})
spotify_df

,track_name,artist,streams
0,Dance Monkey,Tones And I,47797217
1,All I Want for Christmas Is You,Mariah Carey,45828931
2,Last Christmas,Wham!,37725693
3,ROXANNE,Arizona Zervas,32309199
4,Santa Tell Me,Ariana Grande,27506141
...,...,...,...
195,VIBEZ,DaBaby,5560742
196,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,5548309
197,July,Noah Cyrus,5525805
198,QUE PRETENDES,J Balvin,5502509


### Store other CSV into DataFrame, confirmed row count before transformation

In [239]:
#read billboard 100 csv file
billboard_file = "billboard.csv"
billboard_df = pd.read_csv(billboard_file, engine = "python")
billboard_df.count()

Unnamed: 0         97225
Artists            97225
Name               97225
Weekly.rank        97225
Peak.position      90156
Weeks.on.chart     90156
Week               97225
Date               84978
Genre              97225
Writing.Credits    90012
Lyrics             97225
Features           21289
dtype: int64

# Transform billboard_df to match tables

In [240]:
#renamed columns to match sql schemata
billboard_df = billboard_df[["Artists", "Name", "Weekly.rank"]].copy()
billboard_df = billboard_df.rename(columns={'Artists': 'artists', 'Name': 'track_name', 'Weekly.rank': 'weekly_rank'})
billboard_df.head()

,artists,track_name,weekly_rank
0,"Lil Nas,",Old Town Road,1
1,"Shawn Mendes, Camila Cabello",Senorita,2
2,Billie Eilish,Bad Guy,3
3,Khalid,Talk,4
4,"Ed Sheeran, Justin Bieber",I Don't Care,5


In [241]:
billboard_df.head()

,artists,track_name,weekly_rank
0,"Lil Nas,",Old Town Road,1
1,"Shawn Mendes, Camila Cabello",Senorita,2
2,Billie Eilish,Bad Guy,3
3,Khalid,Talk,4
4,"Ed Sheeran, Justin Bieber",I Don't Care,5


In [242]:
# Confirming multiple instances of same song in billboard rankings
billie_eilish = billboard_df.loc[billboard_df['artists'] == 'Billie Eilish']
billie_eilish

,artists,track_name,weekly_rank
2,Billie Eilish,Bad Guy,3
84,Billie Eilish,Ocean Eyes,86
101,Billie Eilish,Bad Guy,3
143,Billie Eilish,When The Party's Over,45
183,Billie Eilish,Ocean Eyes,86
...,...,...,...
3206,Billie Eilish,When The Party's Over,59
3311,Billie Eilish,When The Party's Over,67
3400,Billie Eilish,When The Party's Over,58
3503,Billie Eilish,When The Party's Over,65


In [243]:
# Transformed from 97225 rows to deduplicated 28821 rows. There is now only one instance of each artist's track in the database.

billboard_df_grouped = billboard_df.groupby(['artists', 'track_name']).mean()
billboard_df_grouped.head()


weekly_rank
artists             track_name                   
"Weird Al" Yankovic Canadian Idiot      91.333333
                    Word Crimes         65.000000
'N Sync             Bye Bye Bye         14.260870
                    Gone                27.125000
                    It's Gonna Be Me    21.680000

In [244]:
billboard_df_grouped.reset_index(inplace=True)

In [245]:
# Confirming transformation by viewing unique values for each track. Weekly rank is now average for all weeks on top 100
billie_eilish_grouped = billboard_df_grouped[billboard_df_grouped['artists'] == 'Billie Eilish']
billie_eilish_grouped

,artists,track_name,weekly_rank
636,Billie Eilish,8,79.000000
637,Billie Eilish,All The Good Girls Go To Hell,69.000000
638,Billie Eilish,Bad Guy,5.615385
639,Billie Eilish,Bury A Friend,42.200000
640,Billie Eilish,Come Out And Play,69.000000
641,Billie Eilish,I Love You,53.000000
642,Billie Eilish,Ilomilo,62.000000
643,Billie Eilish,Listen Before I Go,63.000000
644,Billie Eilish,My Strange Addiction,74.250000
645,Billie Eilish,Ocean Eyes,91.833333


In [246]:
#create engine connection to local postgres
rds_connection_string = "postgres:<password>@localhost:5432/music_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [247]:
# Confirm tables
engine.table_names()

['spotify_streams', 'billboard_100']

In [248]:
#load dataframes to postgres server
spotify_df.to_sql(name="spotify_streams",con=engine, if_exists='append', index=False)
billboard_df_grouped.to_sql(name="billboard_100",con=engine, if_exists='append', index=False)

In [249]:
#confirm data is available in server
pd.read_sql_query('select * from spotify_streams', con=engine).head()
pd.read_sql_query('select * from billboard_100', con=engine).head()

,id,artists,track_name,weekly_rank
0,1,"""Weird Al"" Yankovic",Canadian Idiot,91
1,2,"""Weird Al"" Yankovic",Word Crimes,65
2,3,'N Sync,Bye Bye Bye,14
3,4,'N Sync,Gone,27
4,5,'N Sync,It's Gonna Be Me,22
